<a href="https://colab.research.google.com/github/cij1012/pytrorch/blob/master/%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80_mydata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

당뇨병 진행도 예측

* 독립변수

        -age: 나이
        -sex: 성별
        -bmi: BMI(Body mass index)지수
        -bp: 평균혈압
        -s1~s6: 6종류의 혈액검사수치
* 종속변수

        -1년뒤 측정한 당뇨병의 진행률

In [0]:
import pandas as pd
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()

In [3]:
df = pd.DataFrame(diabetes.data, columns = diabetes.feature_names)
df["target"] = diabetes.target
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0


In [4]:
data = df
X,y = data.values[:,:-1],data.values[:,-1:]
print(X.shape,y.shape)

(442, 10) (442, 1)


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [0]:
class MyDataset(Dataset):
  def __init__(self):
    self.x_data = torch.tensor(X, dtype=torch.float)
    self.y_data = torch.tensor(y, dtype=torch.float)
  def __len__(self):
    return len(self.x_data)
  def __getitem__(self,idx):
    x = self.x_data[idx]
    y = self.y_data[idx]
    return x,y

In [0]:
dataset = MyDataset()
dataloader = DataLoader(dataset, batch_size=len(dataset))

In [0]:
model = nn.Linear(10,1)
optimizer = optim.SGD(model.parameters(), lr=0.05)

In [17]:
nb_epochs = 2000
for epoch in range(nb_epochs+1):
  for samples in dataloader:
    x_train, y_train = samples

    prediction = model(x_train)
    cost = F.mse_loss(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch % 100 == 0:
      print('Epoch {:4d}/{} Cost : {:.6f}'.format(epoch, nb_epochs,cost.item()))

Epoch    0/2000 Cost : 29112.013672
Epoch  100/2000 Cost : 5568.070801
Epoch  200/2000 Cost : 5260.198730
Epoch  300/2000 Cost : 4997.226074
Epoch  400/2000 Cost : 4771.980957
Epoch  500/2000 Cost : 4578.471680
Epoch  600/2000 Cost : 4411.687988
Epoch  700/2000 Cost : 4267.441895
Epoch  800/2000 Cost : 4142.229492
Epoch  900/2000 Cost : 4033.117432
Epoch 1000/2000 Cost : 3937.648682
Epoch 1100/2000 Cost : 3853.766602
Epoch 1200/2000 Cost : 3779.742920
Epoch 1300/2000 Cost : 3714.128662
Epoch 1400/2000 Cost : 3655.709229
Epoch 1500/2000 Cost : 3603.458740
Epoch 1600/2000 Cost : 3556.517822
Epoch 1700/2000 Cost : 3514.158203
Epoch 1800/2000 Cost : 3475.769287
Epoch 1900/2000 Cost : 3440.831055
Epoch 2000/2000 Cost : 3408.905273


In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [0]:
reg_model = LinearRegression()
reg_model.fit(diabetes['data'],diabetes['target'])
pred_y = reg_model.predict(diabetes['data'])

In [20]:
mean_squared_error(diabetes['target'],pred_y)

2859.6903987680657